In [5]:
import os
import re
import pandas as pd

# Create Initial Dataframe with all video IDs
dataset_folder = "./standard_dataset"
data_records = []

# Regex pattern to extract Video Id and Title from the filename
filename_pattern = re.compile(r"^(.*)_(.*?)_captions\.txt$")

for file in os.listdir(dataset_folder):
    if file.endswith("_captions.txt"):  # Ensure it's a valid transcript file
        match = filename_pattern.match(file)
        if match:
            video_id, video_title = match.groups()  # Extract Video Id and Title
            file_path = os.path.join(dataset_folder, file)

            # Append data to the list
            data_records.append({
                "Video Title": video_title,
                "Link": f"https://www.youtube.com/watch?v={video_id}",
                "LDA Topics": None,
                "BERTopic Topics": None,
                "HLTA Topics": None
            })

# Convert to DataFrame
topics_per_video_df = pd.DataFrame(data_records)
topics_per_video_df.set_index("Video Title", inplace=True)

topics_per_video_df


,Link,LDA Topics,BERTopic Topics,HLTA Topics
Video Title,,,,
$200 Luxury Beach Hotel in The Philippines ðŸ‡µðŸ‡­,https://www.youtube.com/watch?v=--8n6A8Q6M0,None,None,None
Philippines Army vs Thailand Army,https://www.youtube.com/watch?v=0IMWasj76yU,None,None,None
Tour of The House We Built in The Philippines,https://www.youtube.com/watch?v=1kErCqgIVMk,None,None,None
Top 5 Exotic Foods in the Philippines The BRAVE MUST TRY!,https://www.youtube.com/watch?v=2ftG8JuMzz4,None,None,None
VOCAL COACH REACTS - REGINE VELASQUEZ - Araw-Gabi,https://www.youtube.com/watch?v=2TmagN6RhkI,None,None,None
...,...,...,...,...
"Vi & I Visit the Roxas Night Market, Davao",https://www.youtube.com/watch?v=Z9Uz-NUaCG8,None,None,None
Filipina British Life in UK May pa request si mister!ðŸ¤,https://www.youtube.com/watch?v=ZFpnR0xpdRI,None,None,None
"WILD WINGS DAVAO - BEST CHICKEN WINGS, BILAO FILIPINO FOOD FEAST - DAVAO FOOD - ANUJ GABA",https://www.youtube.com/watch?v=zTcCSyucYqI,None,None,None


### HLTA

### BERTopic

### LDA